<a href="https://colab.research.google.com/github/btaitel/flaming-octo-nemesis/blob/master/refs_copies_immutable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part B: References, copies, built-in collections, and immutability #

_(September 1, 2022)_

_Main topics covered in this note:_

- Variables (names), references, and copies
- Primitive types
- Basic collections, e.g., `list`, `set`, `dict`, `tuple`
- Immutability (and hashability)

## References ##

Variables are _names_ for objects. When the objects are "complex" (not "primitive"), modifications through one name may be visible to others.

To wit:

In [2]:
x = [1, 2, 3, 4, 5]
print("x:", x)

y = x
print("y:", y)

y[2] *= -1
print("Modified y:", y)

x: [1, 2, 3, 4, 5]
y: [1, 2, 3, 4, 5]
Modified y: [1, 2, -3, 4, 5]


**Question:** What is `x`?

In [3]:
print(x) # What does this produce?

[1, 2, -3, 4, 5]


In [4]:
%%html
<iframe width="800" height="250" frameborder="0" src="https://pythontutor.com/iframe-embed.html#code=x%20%3D%20%5B1,%202,%203,%204,%205%5D%0Ay%20%3D%20x%0Ay%5B2%5D%20*%3D%20-1&codeDivHeight=400&codeDivWidth=350&cumulative=false&curInstr=0&heapPrimitives=nevernest&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false"> </iframe>

**What's your alternative?** If you really do need a copy, what are your options?

```python
y = [1, 2, 3, 4, 5]
y = x.copy()
y = [e for e in x]
```

Try these in the Python Tutor.

**A tricky case.**

In [5]:
x = [1, 2, ['a', 'b', 'c'], 4, 5]
y = x.copy()
print(y)

[1, 2, ['a', 'b', 'c'], 4, 5]


In [6]:
y[2].append('w')
print(y)

[1, 2, ['a', 'b', 'c', 'w'], 4, 5]


In [7]:
print(x) # What is the result?

[1, 2, ['a', 'b', 'c', 'w'], 4, 5]


In Python, all unique objects have an _identifier_ associated with them. You can query these.

In [8]:
id(x), id(y)

(135036471199104, 135036474014080)

In [9]:
id(x[2]), id(y[2])

(135036471198912, 135036471198912)

In this case, `x` and `y` are distinct objects, but `x[2]` and `y[2]` refer to the same object. When we "copied" `x[2]` into `y[2]`, we copied the `id(x[2])` rather than duplicating the entire object. This kind of copy is sometimes called a _shallow copy_.

Still not clear? Check out a Python Tutor version.

In [10]:
%%html

<iframe width="1024" height="350" frameborder="0" src="https://pythontutor.com/iframe-embed.html#code=x%20%3D%20%5B1,%202,%20%5B'a',%20'b',%20'c'%5D,%204,%205%5D%0Ay%20%3D%20x.copy%28%29%0Ay%5B2%5D.append%28'w'%29&codeDivHeight=400&codeDivWidth=350&cumulative=false&curInstr=0&heapPrimitives=nevernest&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false"> </iframe>

**What if you really need a copy for a nested data structure?** The preceding example illustrates that `.copy()` performs a _shallow_ copy. But what if you want a non-shallow, or _deep_, copy? There's a module for that!

In [11]:
from copy import deepcopy

print('x:', x)
z = deepcopy(x)
print('z:', z)

print('=== appending ===')
z[2].append('@')
print('x:', x)
print('z:', z)

x: [1, 2, ['a', 'b', 'c', 'w'], 4, 5]
z: [1, 2, ['a', 'b', 'c', 'w'], 4, 5]
=== appending ===
x: [1, 2, ['a', 'b', 'c', 'w'], 4, 5]
z: [1, 2, ['a', 'b', 'c', 'w', '@'], 4, 5]


**Exercise** (taken from Notebook 1). Let `L` be a list of strings, e.g.,

```python
L = ['abc', 'def', 'ghi']
```

Complete the function, `rev_str_cat_list(L)` so that it reverses the elements in the list and then concatenates these strings into a single string. It should not modify `L`.

For instance, `rev_str_cat_list(L)` on the above list would return,

```python
'ghidefabc'
```

Your friend supplies the following solution. It appears to produce the correct result, but is wrong. Why?

In [12]:
def rev_str_cat_list(L):
    L.reverse()
    return ''.join(L)

L = ['abc', 'def', 'ghi']
result = rev_str_cat_list(L)
print(repr(result)) # So right, and yet so wrong. Why?

'ghidefabc'


> _Answer:_ This function is considered _incorrect_ because it modifies its input. Try `print(L)` after the call to `rev_str_cat_list(L)` to verify this claim.
>
> In this case, the exercise stipulates that the function should not modify its input. However, you should always _assume_ that convention unless told otherwise. Why? Remember that you are writing code for others. By adhering to the convention that functions do not modify their inputs, it makes it easier for others to reason about the behavior of your code. When we want your function to modify its input, we will tell you to do so.

## Data structure costs ##

One thing you need to be doing right now is trying to understand the different elementary data structures that Python offers and their tradeoffs.

In this part of the course, the ones you should learn are:

**Tuples** store a _fixed-length_, _ordered sequence_ of values. You can quickly look up any element by its _position_ or _index_. It is "immutable" in the sense that you cannot add or remove elements from the sequence.

**Lists** store a _variable-length_, _ordered sequence_ of values. You can quickly look up any element by its _position_ (or _index_) in the sequence. You can also add or remove elements from the sequence.

**Dictionaries** store key-value pairs. You decide what keys go with what values, where the keys must be unique. You can then quickly look up any value by its _key_.

**Sets** store unique values. They are similar to mathematical sets. You can quickly check whether a value is in the set and perform operations like set intersections, set unions, and so on.

To get a better feel the tradeoffs, let's do a series of experiments. For these experiments, we'll start with a list of 100,000 integer values drawn uniformly from the interval [0, 1 billion].

In [15]:
from random import randint

n = 100_000
L = [randint(0, 1_000_000_000) for _ in range(n)]

# Step 4 — Why it’s slow for a list
# Lists check in by going through each element one by one (O(n) time).
# So longer lists → slower.
# If you used a set, it would be much faster because in is almost instant (O(1)).

Suppose I want to check whether the number `50` is in the list `L`. I can do that using the expression, `50 in L`. Let's see how long it takes.

In [14]:
%timeit 50 in L

#it takes around 1 ms to see if 50 is in this list

1.09 ms ± 6.64 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Suppose I store these values instead in a _set_, `S`. How long does lookup take?

In [18]:
S = set(L)
print(f'{len(L):,} vs. {len(S):,}') # `len(S)` should be close to `len(L)`

#basically we are turning a list into a set, we had 5 duplicates
#but its show how much faster it is to determine membership with the in keyword
#in a set vs a list

100,000 vs. 99,995


In [19]:
%timeit 50 in S

#32 nanoseconds is 31,250 times faster than 1 millisecond.

# 1 millisecond = 1 ms = 1 times 10^{-3} seconds
# 32 nanoseconds = 32 ns = 1 times 10^{-9} seconds


32.9 ns ± 9.85 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Lastly, let's use a dictionary. We'll use the integers themselves as keys and associate them with a dummy value, `True`.

In [20]:
D = {k: True for k in L}
%timeit 50 in D

#basically same efficiency as a finding membership in a set

48.5 ns ± 13.8 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


## Immutability ##

Certain types of objects in Python are _immutable_, meaning they cannot be modified once they are created. Integers, floating-point values, strings, and tuples are immutable.

Example:

In [21]:
s = 'abc'
print(s[1])

b


In [ ]:
#s[1] = 'x'   # Uncomment this code and see what happens

In [24]:
t = (1, 'a', 3)

# t[1] = '@'   # Uncomment: what happens?

Why is mutability important? The _values_ of a set or the _keys_ of a dictionary _must_ be immutable. Immutability is what allows lookups in those data structures to be fast. (Why?)

In [25]:
D = {'a': 1, 'b': 2, 'c': 3}
print(D)

{'a': 1, 'b': 2, 'c': 3}


In [26]:
D[(1, 'a', 3)] = 3.14159 #remember with dictionarie we dont need to explicilty append
#or 'add'. you just simply specifyt the new kv pair
print(D)

{'a': 1, 'b': 2, 'c': 3, (1, 'a', 3): 3.14159}


In [27]:
D[(1, 'a', 3)]

3.14159

In [28]:
{1, 'a', 3} # a set

{1, 3, 'a'}

In [30]:
#D[{1, 'a', 3}] = 2.71828  # Uncomment: what happens?

# we get type error that set is unhashable
# which means that set cannot be a key in a dict bc sets are mutable

**Qualifications.** Two statements above need some qualification.

First, when you use a mutable type as a dictionary key, you'll trigger an error that uses the term "unhashable" rather than "mutable." Immutability and hashability are distinct concepts: immutable objects have values that cannot be modified, whereas hashable objects are simply any object that has a procedure associated with it for converting it into an integer. It is possible for an immutable object to be unhashable, and for a hashable object to be mutable. **However,** _almost_ every immutable type is hashable by convention. (Even the Python documentation on [dictionaries](https://docs.python.org/3/library/stdtypes.html#typesmapping) has some confusing language on this point, referring to both hashability and mutability in the same breath.)

Second, a tuple is immutable under a narrow definition of what its "value" is considered to be. For instance, consider the following code. Based on its output, would you say that a tuple is "immutable" under a common-sense definition?

In [31]:
immutable_in_air_quotes = (1, ['a', 'b', 'c'], 3)
immutable_in_air_quotes

(1, ['a', 'b', 'c'], 3)

In [32]:
immutable_in_air_quotes[1].append('@')
immutable_in_air_quotes

#basically tuples are immutabel but inside there is a mutable type and we can still mutate it

(1, ['a', 'b', 'c', '@'], 3)

The "narrow definition" of its value is that `id` applied to each component is always the same once you create it. In particular, you cannot change what object exists in each component:

In [33]:
print(list(id(e) for e in immutable_in_air_quotes))

#each object in this tuple has a specific id, and those ids cannot be mutated cause its a tuple
#however when we change an list element inside a tuple, its fine cause its not
#altering the ids within the tuple itself

[11654376, 135036050044800, 11654440]


In [ ]:
#immutable_in_air_quotes[1] = ['different', 'list', 'no', 'way']  # Uncomment: Will error-out

So, since component 1 of `immutable_in_air_quotes` is a list, which is mutable, you can modify it. But the list that you put there can never be replaced by another list.

In [ ]:
# L = immutable_in_air_quotes[1]

# # remove old items
# L.clear()

# # append new items
# L.extend(['different', 'list', 'no', 'way'])

#chatgpt basically told me you could do it like this
#clear, then extend methods

## Summary ##

1. Every distinct object in Python has an ID, which you can see by `id(x)` for the object `x`.

2. An assignment _copies_ these IDs. That is, in the assignment `y = x`, it will be the case that `id(y)` equals `id(x)`.

3. The built-in primitive types, which are `bool`, `int`, `float`, and `str`, are immutable and effectively copied on assignment. That's because the `id` of any two values is equal if the values are also equal.

In [34]:
x = 5
y = 5
id(x), id(y)

#primitive types basically have the same id even if you dont copy one to the other

(11654504, 11654504)

In [35]:
x = [1, 2, 3]
y = [1, 2, 3]
id(x), id(y)

#lists by default are mutable and take on diff ids when created

(135036049571328, 135036121813760)

4. Shallow vs. deep copies: An object's `.copy()` function will perform a shallow copy. For deep copies, use `deepcopy` from the `copy` module.

5. Python provides several built-in _collections_ or _containers_ for holding a bunch of objects. These are `list`, `set`, `dict`, and `tuple`. You should learn the distinction and start building some intuition for when you might use one or another.

6. For lookups, the `set` and `dict` (for keys) collections can do that quickly. They do so by restricting what you can store in them, namely, _immutable_ objects.